In [47]:
import pandas as pd

In [48]:
# Baca data kunci jawaban
kunci_jawaban_path = 'kuncijawaban.csv'
kunci_jawaban_df = pd.read_csv(kunci_jawaban_path)

# Baca data jawaban siswa
jawaban_siswa_path = 'jawabansiswa.csv'
jawaban_siswa_df = pd.read_csv(jawaban_siswa_path)

In [49]:
# Function to calculate the
# Jaro Similarity of two strings
def jaro_distance(s1, s2) :

    # If the strings are equal
    if (s1 == s2) :
        return 1.0;

    # Length of two strings
    len1 = len(s1);
    len2 = len(s2);

    if (len1 == 0 or len2 == 0) :
        return 0.0;

    # Maximum distance upto which matching
    # is allowed
    max_dist = (max(len(s1), len(s2)) // 2 ) - 1;

    # Count of matches
    match = 0;

    # Hash for matches
    hash_s1 = [0] * len(s1) ;
    hash_s2 = [0] * len(s2) ;

    # Traverse through the first string
    for i in range(len1) :

        # Check if there is any matches
        for j in range( max(0, i - max_dist),
                    min(len2, i + max_dist + 1)) :

            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0) :
                hash_s1[i] = 1;
                hash_s2[j] = 1;
                match += 1;
                break;

    # If there is no match
    if (match == 0) :
        return 0.0;

    # Number of transpositions
    t = 0;

    point = 0;

    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1) :
        if (hash_s1[i]) :

            # Find the next matched character
            # in second string
            while (hash_s2[point] == 0) :
                point += 1;

            if (s1[i] != s2[point]) :
                point += 1;
                t += 1;
            else :
                point += 1;

        t /= 2;

    # Return the Jaro Similarity
    return ((match / len1 + match / len2 +
            (match - t) / match ) / 3.0);

# Jaro Winkler Similarity
def jaro_Winkler(s1, s2) :

    jaro_dist = jaro_distance(s1, s2);

    # If the jaro Similarity is above a threshold
    if (jaro_dist > 0.7) :

        # Find the length of common prefix
        prefix = 0;

        for i in range(min(len(s1), len(s2))) :

            # If the characters match
            if (s1[i] == s2[i]) :
                prefix += 1;

            # Else break
            else :
                break;

        # Maximum of 4 characters are allowed in prefix
        prefix = min(4, prefix);

        # Calculate jaro winkler Similarity
        jaro_dist += 0.1 * prefix * (1 - jaro_dist);

    return jaro_dist;

In [55]:
results = []

# Looping untuk setiap baris jawaban siswa
for index, row in jawaban_siswa_df.iterrows():
    nama = row['Nama']
    jawaban_soal1 = row['JawabanSoal1']
    jawaban_soal2 = row['JawabanSoal2']
    jawaban_soal3 = row['JawabanSoal3']

    # Ganti variabel jawaban_siswa menjadi array
    jawaban_siswa = [jawaban_soal1, jawaban_soal2, jawaban_soal3]

    nilai = 0  # Inisialisasi nilai siswa

    # Looping untuk setiap baris kunci jawaban
    for kunci_index, kunci_row in kunci_jawaban_df.iterrows():
        kunci_panjang = kunci_row['KunciJawaban1']
        kunci_pendek = kunci_row['KunciJawaban2']

        # Cocokkan jawaban siswa dengan kunci jawaban menggunakan Jaro-Winkler
        similarity_soal_panjang = jaro_Winkler(jawaban_siswa[kunci_index], kunci_panjang)
        similarity_soal_pendek = jaro_Winkler(jawaban_siswa[kunci_index], kunci_pendek)

        # Hitung nilai untuk setiap soal
        nilai_soal = max(similarity_soal_panjang, similarity_soal_pendek)

        # Tambahkan nilai soal ke nilai total siswa
        nilai += nilai_soal

    # Tampilkan hasil nilai untuk setiap siswa
    nilai = round((nilai / kunci_jawaban_df['ID'].count()) * 100)
    results.append({'Nama': nama, 'Jawaban_Soal': jawaban_siswa, 'Nilai': nilai})

# Create a DataFrame from the results list
result_df = pd.DataFrame(results)

result_df

,Nama,Jawaban_Soal,Nilai
0,Luthfi Rahman,[Letak geografis diantara samudra hindia dan p...,89
1,Agoes Budi,[Terletak di antara dua samudra dan dua benua....,90
2,Bagus Febri,"[Berdasarkan letak geografis, ASEAN terletak d...",89


In [56]:
# Save the DataFrame to a CSV file
result_df.to_csv('hasil_nilai.csv', index=False)